# 🌩️ **CloudLeecher: Production Backend**

Welcome to the **CloudLeecher** backend. This notebook turns your Google Colab instance into a powerful, high-speed torrent downloader that saves files directly to your Google Drive.

### **Instructions**
1.  **Mount Drive**: Connect your Google storage.
2.  **Install**: Set up the environment.
3.  **Start Services**: Launch the backend and get your public connection URL.
4.  **Connect**: Paste the URL into the CloudLeecher Frontend.

## 1. 📂 **Mount Google Drive**
We need access to your Drive to save the downloaded files.

In [1]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define and Create Download Directory
DOWNLOAD_DIR = "/content/drive/MyDrive/TorrentDownloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

print(f"✅ Download Directory Ready: {DOWNLOAD_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Download Directory Ready: /content/drive/MyDrive/TorrentDownloads


## 2. 🛠️ **Install Dependencies**
Installing `aria2` (the download engine), `flask` (the API server), and `pyngrok` (for public access).

In [2]:
%%capture
!apt-get update -qq
!apt-get install -y -qq aria2
!pip install -q flask flask-cors pyngrok

print("✅ All dependencies installed successfully.")

## 3. 🚀 **Start Downloader Service**
Initializing the Aria2 RPC server in the background.

In [3]:
import subprocess
import os

# Define local temporary directory
TEMP_DOWNLOAD_DIR = "/content/temp_downloads"
os.makedirs(TEMP_DOWNLOAD_DIR, exist_ok=True)

# Start Aria2c as a daemon process
cmd = [
    "aria2c",
    "--enable-rpc",
    "--rpc-listen-all=true",
    "--rpc-allow-origin-all",
    f"--dir={TEMP_DOWNLOAD_DIR}",
    "--file-allocation=none",
    "--max-connection-per-server=16",
    "--split=16",
    "--min-split-size=1M",
    "--seed-time=0",
    "--daemon=true"
]

subprocess.run(
    cmd,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

print("✅ Aria2 Background Service Started (Staging: /content/temp_downloads).")

✅ Aria2 Background Service Started.


## 4. 📝 **Create API Backend**
Generating the `app.py` file which serves as the brain of CloudLeecher.

In [4]:
%%writefile app.py
import xmlrpc.client
from flask import Flask, request, jsonify, abort
from flask_cors import CORS
import os
import shutil
import base64
import json
from datetime import datetime
from collections import deque
import threading
import time
import secrets
from functools import wraps

app = Flask(__name__)
CORS(app)

# Configuration
# Use a local temporary directory for downloading to avoid FUSE issues
TEMP_DOWNLOAD_DIR = "/content/temp_downloads"
# Final destination on Google Drive
FINAL_DIR = "/content/drive/MyDrive/TorrentDownloads"

ARIA2_RPC_URL = "http://localhost:6800/rpc"
LOG_FILE = "/content/backend_logs.json"

# Ensure directories exist
os.makedirs(TEMP_DOWNLOAD_DIR, exist_ok=True)
os.makedirs(FINAL_DIR, exist_ok=True)

# Authentication
API_KEY = os.environ.get('CLOUDLEECHER_API_KEY')
if not API_KEY:
    # If not provided via env, generate one (fallback)
    API_KEY = secrets.token_hex(16)
    print(f"\n{'='*50}\nGenerated API Key: {API_KEY}\n{'='*50}\n")
else:
    print(f"\n{'='*50}\nUsing Configured API Key: {API_KEY}\n{'='*50}\n")

# In-memory log storage (last 100 entries)
logs = deque(maxlen=100)

# Connect to Aria2 RPC
s = xmlrpc.client.ServerProxy(ARIA2_RPC_URL)

def log(level, operation, message, gid=None, extra=None):
    """Add entry to log with timestamp and details"""
    entry = {
        "timestamp": datetime.now().isoformat(),
        "level": level,  # info, warning, error
        "operation": operation,
        "message": message,
        "gid": gid,
        "extra": extra
    }
    logs.append(entry)
    
    # Also write to file for persistence
    try:
        with open(LOG_FILE, 'a') as f:
            f.write(json.dumps(entry) + '\n')
    except:
        pass
    
    # Print to console
    print(f"[{level.upper()}] {operation}: {message}" + (f" (GID: {gid})" if gid else ""))

def check_auth(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if request.method == 'OPTIONS':
            return f(*args, **kwargs)
            
        request_key = request.headers.get('x-api-key')
        if not request_key or request_key != API_KEY:
            log("warning", "auth", "Unauthorized access attempt")
            return jsonify({"error": "Unauthorized: Invalid or missing API Key"}), 401
        return f(*args, **kwargs)
    return decorated_function

class BackgroundMonitor(threading.Thread):
    def __init__(self):
        super().__init__()
        self.daemon = True
        self.stopped = False

    def run(self):
        log("info", "monitor", "Background monitor started")
        while not self.stopped:
            try:
                self.check_completed_downloads()
            except Exception as e:
                pass # Silent fail (e.g. aria2 not ready yet)
            time.sleep(5)

    def check_completed_downloads(self):
        try:
            # Get stopped tasks (includes complete, error, removed)
            stopped = s.aria2.tellStopped(0, 100, ["gid", "status", "files", "bittorrent", "totalLength", "errorCode"])
            
            for task in stopped:
                status = task['status']
                gid = task['gid']
                
                if status == 'complete':
                    self.handle_complete_task(task)
                elif status == 'error':
                     # Log error and remove
                     err_code = task.get('errorCode', 'unknown')
                     log("error", "download", f"Task failed with error code {err_code}", gid=gid)
                     try: s.aria2.removeDownloadResult(gid)
                     except: pass
                elif status == 'removed':
                     try: s.aria2.removeDownloadResult(gid)
                     except: pass
                     
        except Exception as e:
            pass

    def handle_complete_task(self, task):
        gid = task['gid']
        files = task.get('files', [])
        
        if not files:
            try: s.aria2.removeDownloadResult(gid)
            except: pass
            return

        # Try to find the root file or directory
        bt_name = task.get('bittorrent', {}).get('info', {}).get('name')
        
        source_path = None
        
        # Strategy 1: Look for the name provided in metadata
        if bt_name:
             possible_path = os.path.join(TEMP_DOWNLOAD_DIR, bt_name)
             if os.path.exists(possible_path):
                 source_path = possible_path
        
        # Strategy 2: Use the first file path
        if not source_path and files:
            first_file_path = files[0]['path']
            # If the path starts with TEMP_DOWNLOAD_DIR, great
            if os.path.exists(first_file_path):
                # If it's a multi-file torrent, first_file_path is deep inside
                # We want the top-level directory.
                rel_path = os.path.relpath(first_file_path, TEMP_DOWNLOAD_DIR)
                top_level = rel_path.split(os.sep)[0]
                possible_path = os.path.join(TEMP_DOWNLOAD_DIR, top_level)
                if os.path.exists(possible_path):
                    source_path = possible_path

        if not source_path:
             log("error", "move", "Could not locate downloaded files", gid=gid)
             # Don't remove result so we can debug? Or remove to avoid loop?
             # Remove it to avoid infinite loop
             try: s.aria2.removeDownloadResult(gid)
             except: pass
             return

        dest_path = os.path.join(FINAL_DIR, os.path.basename(source_path))
        
        # Handle collision
        if os.path.exists(dest_path):
            base, ext = os.path.splitext(os.path.basename(source_path))
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            dest_path = os.path.join(FINAL_DIR, f"{base}_{timestamp}{ext}")

        log("info", "move", f"Moving {os.path.basename(source_path)} to Drive...", gid=gid)
        
        try:
            shutil.move(source_path, dest_path)
            log("info", "move", f"Move successful: {os.path.basename(dest_path)}", gid=gid)
            try: s.aria2.removeDownloadResult(gid)
            except: pass
        except Exception as e:
            log("error", "move", f"Move failed: {str(e)}", gid=gid)
            # If move failed (e.g. Drive full), we leave the task in 'stopped' state
            # but 'tellStopped' returns it again.
            # To avoid retry loop spam, we might need to blacklist it in memory
            # or just log error. 
            # For now, we leave it.

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "service": "CloudLeecher-Backend"})

@app.route('/api/logs', methods=['GET'])
@check_auth
def get_logs():
    try:
        return jsonify({"logs": list(logs)})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/download/magnet', methods=['POST'])
@check_auth
def add_magnet():
    data = request.json
    magnet_link = data.get('magnet')
    if not magnet_link:
        log("error", "add_magnet", "Magnet link is required")
        return jsonify({"error": "Magnet link is required"}), 400
    
    active = s.aria2.tellActive(["gid", "status"])
    waiting = s.aria2.tellWaiting(0, 100, ["gid", "status"])
    
    if len(active) > 0 or len(waiting) > 0:
        log("warning", "add_magnet", f"Rejected: {len(active)} active, {len(waiting)} waiting tasks already exist")
        return jsonify({"error": "Another download is already in progress. Please wait for it to complete."}), 429
    
    try:
        gid = s.aria2.addUri([magnet_link])
        log("info", "add_magnet", "Magnet link added successfully", gid=gid, extra={"magnet": magnet_link[:50] + "..."})
        return jsonify({"status": "success", "gid": gid})
    except Exception as e:
        log("error", "add_magnet", f"Failed: {str(e)}", extra={"magnet": magnet_link[:50] + "..."})
        return jsonify({"error": str(e)}), 500

@app.route('/api/download/file', methods=['POST'])
@check_auth
def add_torrent_file():
    try:
        data = request.json
        b64_content = data.get('torrent')
        if not b64_content:
            log("error", "add_torrent_file", "Torrent file content is required")
            return jsonify({"error": "Torrent file content is required"}), 400

        active = s.aria2.tellActive(["gid", "status"])
        waiting = s.aria2.tellWaiting(0, 100, ["gid", "status"])
        
        if len(active) > 0 or len(waiting) > 0:
            log("warning", "add_torrent_file", f"Rejected: {len(active)} active, {len(waiting)} waiting tasks already exist")
            return jsonify({"error": "Another download is already in progress. Please wait for it to complete."}), 429

        raw_bytes = base64.b64decode(b64_content)
        binary_torrent = xmlrpc.client.Binary(raw_bytes)
        
        log("info", "add_torrent_file", f"Received torrent file ({len(raw_bytes)} bytes), adding to aria2...")
        gid = s.aria2.addTorrent(binary_torrent)
        log("info", "add_torrent_file", "Torrent file added successfully, downloading metadata...", gid=gid)
        
        try:
            status = s.aria2.tellStatus(gid, ["gid", "status", "files", "bittorrent"])
            torrent_name = status.get('bittorrent', {}).get('info', {}).get('name', 'Unknown')
            log("info", "add_torrent_file", f"Torrent name: {torrent_name}", gid=gid, extra={"status": status.get('status')})
        except:
            pass
        
        return jsonify({"status": "success", "gid": gid})
    except Exception as e:
        log("error", "add_torrent_file", f"Failed: {str(e)}")
        return jsonify({"error": str(e)}), 500

@app.route('/api/status', methods=['GET'])
@check_auth
def get_status():
    try:
        basic_keys = ["gid", "status", "totalLength", "completedLength", "downloadSpeed", "uploadSpeed", "dir", "files", "errorMessage", "errorCode", "followedBy", "following"]
        extended_keys = basic_keys + ["numSeeders", "connections", "infoHash", "bittorrent"]
        
        active = s.aria2.tellActive(extended_keys)
        waiting = s.aria2.tellWaiting(0, 100, basic_keys)
        stopped = s.aria2.tellStopped(0, 100, basic_keys)
        
        return jsonify({
            "active": active,
            "waiting": waiting,
            "stopped": stopped
        })
    except Exception as e:
        log("error", "get_status", f"Failed: {str(e)}")
        return jsonify({"error": str(e)}), 500

@app.route('/api/control/pause', methods=['POST'])
@check_auth
def pause_download():
    try:
        gid = request.json.get('gid')
        s.aria2.pause(gid)
        log("info", "pause_download", "Download paused", gid=gid)
        return jsonify({"status": "paused", "gid": gid})
    except Exception as e:
        log("error", "pause_download", f"Failed: {str(e)}", gid=request.json.get('gid'))
        return jsonify({"error": str(e)}), 500

@app.route('/api/control/resume', methods=['POST'])
@check_auth
def resume_download():
    try:
        gid = request.json.get('gid')
        s.aria2.unpause(gid)
        log("info", "resume_download", "Download resumed", gid=gid)
        return jsonify({"status": "resumed", "gid": gid})
    except Exception as e:
        log("error", "resume_download", f"Failed: {str(e)}", gid=request.json.get('gid'))
        return jsonify({"error": str(e)}), 500

@app.route('/api/control/remove', methods=['POST'])
@check_auth
def remove_download():
    try:
        gid = request.json.get('gid')
        s.aria2.forceRemove(gid)
        log("info", "remove_download", "Download removed", gid=gid)
        return jsonify({"status": "removed", "gid": gid})
    except xmlrpc.client.Fault as e:
        if 'not found' in str(e).lower():
            log("info", "remove_download", "GID not found (already removed)", gid=request.json.get('gid'))
            return jsonify({"status": "removed", "gid": request.json.get('gid')})
        else:
            log("error", "remove_download", f"Aria2 error: {str(e)}", gid=request.json.get('gid'))
            return jsonify({"error": str(e)}), 500
    except Exception as e:
        log("error", "remove_download", f"Failed: {str(e)}", gid=request.json.get('gid'))
        return jsonify({"error": str(e)}), 500

@app.route('/api/drive/info', methods=['GET'])
@check_auth
def drive_info():
    try:
        total, used, free = shutil.disk_usage(FINAL_DIR)
        return jsonify({
            "total": total,
            "used": used,
            "free": free
        })
    except Exception as e:
        return jsonify({"total": 0, "used": 0, "free": 0})

@app.route('/api/cleanup', methods=['POST'])
@check_auth
def cleanup_all():
    """Nuclear option: Remove ALL tasks from aria2 and start fresh"""
    try:
        active = s.aria2.tellActive(["gid"])
        waiting = s.aria2.tellWaiting(0, 9999, ["gid"])
        stopped = s.aria2.tellStopped(0, 9999, ["gid"])
        
        removed_count = 0
        
        for task in active + waiting:
            try:
                s.aria2.forceRemove(task['gid'])
                removed_count += 1
            except:
                pass
        
        try:
            s.aria2.purgeDownloadResult()
            removed_count += len(stopped)
        except:
            pass
        
        log("info", "cleanup_all", f"Cleaned up {removed_count} tasks")
        return jsonify({"status": "success", "removed": removed_count})
    except Exception as e:
        log("error", "cleanup_all", f"Failed: {str(e)}")
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    log("info", "startup", "CloudLeecher Backend starting...")
    
    # Start background monitor
    monitor = BackgroundMonitor()
    monitor.start()
    
    app.run(port=5000)

Overwriting app.py


## 5. 🌐 **Launch Public Server**
Starting the application and generating your public access URL.

> **⚠️ Important**: Ensure you have added your Ngrok Authtoken to Colab Secrets with the key `NGROK-AUTHTOKEN`.

In [ ]:
from pyngrok import ngrok
from google.colab import userdata
import subprocess
import sys
import time
import os
import secrets

# 1. Authenticate Ngrok
try:
    AUTH_TOKEN = userdata.get("NGROK-AUTHTOKEN")
    ngrok.set_auth_token(AUTH_TOKEN)
except Exception as e:
    print("❌ Error: Ngrok Auth Token not found! Please add 'NGROK-AUTHTOKEN' to Colab Secrets (Key icon on the left).")
    raise e

# 2. Cleanup Old Processes (Port 5000)
ngrok.kill()
os.system("fuser -k 5000/tcp > /dev/null 2>&1")

# 3. Generate API Key
api_key = secrets.token_hex(16)
os.environ['CLOUDLEECHER_API_KEY'] = api_key

# 4. Start Flask App in Background
log_file = open("flask.log", "w")
env = os.environ.copy()
subprocess.Popen([sys.executable, "app.py"], stdout=log_file, stderr=log_file, env=env)
time.sleep(3)  # Allow Flask to initialize

# 5. Open Ngrok Tunnel
try:
    public_url = ngrok.connect(5000).public_url
    print("\n" + "="*60)
    print(f"🔗 PUBLIC URL: {public_url}")
    print(f"🔑 API KEY:    {api_key}")
    print("="*60 + "\n")
    print("✅ CloudLeecher Backend is Online!")
    print("🌍 Frontend App: https://cloudleecher.web.app")
    print("📋 Copy the URL and API KEY above and paste them into the CloudLeecher Frontend app.")

    # Keep cell running to keep thread alive
    while True:
        time.sleep(10)
except Exception as e:
    print(f"❌ Failed to start Ngrok: {e}")


🔗 PUBLIC URL: https://vitalistically-falsifiable-donnette.ngrok-free.dev

✅ CloudLeecher Backend is Online!
🌍 Frontend App: https://cloudleecher.web.app
📋 Copy the URL above (PUBLIC URL) and paste it into the CloudLeecher Frontend app.
